In [5]:
import numpy as np
import torch
from torch import Tensor
from tgb.nodeproppred.dataset import NodePropPredDataset
from tgb.nodeproppred.dataset_pyg import PyGNodePropPredDataset
from tgb.nodeproppred.evaluate import Evaluator

In [6]:
name = "tgbn-trade"
dataset = NodePropPredDataset(name=name, root = r"data/", preprocess=True)
data = dataset.full_data

dataset_py = PyGNodePropPredDataset(name=name, root=r"data/")

raw file found, skipping download
Dataset directory is  /home/joe/anaconda3/lib/python3.11/site-packages/tgb/data/tgbn_trade
loading processed file
raw file found, skipping download
Dataset directory is  /home/joe/anaconda3/lib/python3.11/site-packages/tgb/data/tgbn_trade
loading processed file


In [7]:
df, node_label_dict, edge_feat = dataset.generate_processed_files()

loading processed file


In [8]:
node_label_dict[2016][2]

array([1.66176749e-05, 1.71066150e-04, 4.51711481e-04, 4.73375773e-05,
       0.00000000e+00, 1.31982385e-04, 6.80742975e-06, 5.28580412e-03,
       3.92755681e-04, 4.45800180e-03, 6.90175903e-05, 4.18193144e-06,
       8.43571034e-03, 1.01393632e-01, 6.98250489e-03, 3.55187787e-02,
       3.18078334e-04, 2.22475608e-04, 2.30559627e-03, 7.06714970e-04,
       2.47887916e-03, 8.92382821e-02, 0.00000000e+00, 1.89020157e-04,
       1.59501695e-02, 3.92299756e-04, 1.21008274e-02, 1.99166576e-01,
       1.48254186e-05, 0.00000000e+00, 2.12555312e-05, 1.57215468e-06,
       0.00000000e+00, 1.02954121e-03, 2.04851754e-05, 3.11286626e-05,
       4.39561241e-02, 7.40380933e-02, 1.03799940e-03, 9.89317633e-03,
       2.68653250e-02, 1.04395787e-03, 6.13140323e-07, 5.90092536e-03,
       5.92702313e-06, 1.60491995e-02, 7.84347968e-05, 2.62392615e-05,
       2.48970659e-02, 3.65060604e-03, 0.00000000e+00, 1.07378164e-05,
       1.55512195e-02, 1.09437687e-04, 2.40228379e-03, 1.51874544e-02,
      

In [ ]:
evaluator = Evaluator(name=name)
metric = dataset.eval_metric

In [13]:
a, b = np.unique(dataset.test_mask, return_counts=True)

In [35]:
data_py = dataset_py.get_TemporalData()
data_py

TemporalData(src=[468245], dst=[468245], t=[468245], msg=[468245, 1], y=[468245])

In [48]:
_, _, label = dataset_py.get_node_label(1988)
label.shape

torch.Size([205, 255])

In [11]:
data

TemporalData(src=[468245], dst=[468245], t=[468245], msg=[468245, 1], y=[468245])

In [29]:
np.unique(np.vstack((data["sources"], data["destinations"]))).shape

(255,)

In [3]:
# Example shapes
evaluator = Evaluator(name="tgbn-trade")
y_true = np.array([3, 2, 1, 0])  # True relevance scores
y_pred = np.array([0.9, 0.8, 0.1, 0.0])  # Predicted scores

eval_method = {"y_pred": y_pred, 
                       "y_true": y_true, 
                       "eval_metric": ["rmse"]}
result = evaluator.eval(eval_method)

In [5]:
result

{}

In [6]:
class PersistantForecaster:
    def __init__(self, num_class):
        self.dict = {}
        self.num_class = num_class

    def update_dict(self, node_id, label):
        self.dict[node_id] = label

    def query_dict(self, node_id):
        r"""
        Parameters:
            node_id: the node to query
        Returns:
            returns the last seen label of the node if it exists, if not return zero vector
        """
        if node_id in self.dict:
            return self.dict[node_id]
        else:
            return np.zeros(self.num_class)


In [9]:
"""
implement persistant forecast as baseline for the node prop pred task
simply predict last seen label for the node
"""

import timeit
import numpy as np
from torch_geometric.loader import TemporalDataLoader

# local imports
from tgb.nodeproppred.dataset_pyg import PyGNodePropPredDataset
from tgb.nodeproppred.evaluate import Evaluator

device = "cpu"

name = "tgbn-trade"
dataset = PyGNodePropPredDataset(name=name, root="datasets")
num_classes = dataset.num_classes
data = dataset.get_TemporalData()
data = data.to(device)

eval_metric = dataset.eval_metric
forecaster = PersistantForecaster(num_classes)
evaluator = Evaluator(name=name)


# Ensure to only sample actual destination nodes as negatives.
min_dst_idx, max_dst_idx = int(data.dst.min()), int(data.dst.max())
train_data, val_data, test_data = data.train_val_test_split(
    val_ratio=0.15, test_ratio=0.15
)

batch_size = 200

train_loader = TemporalDataLoader(train_data, batch_size=batch_size)
val_loader = TemporalDataLoader(val_data, batch_size=batch_size)
test_loader = TemporalDataLoader(test_data, batch_size=batch_size)


def test_n_upate(loader):
    label_t = dataset.get_label_time()  # check when does the first label start
    num_label_ts = 0
    total_score = 0

    for batch in loader:
        batch = batch.to(device)
        src, pos_dst, t, msg = batch.src, batch.dst, batch.t, batch.msg

        query_t = batch.t[-1]
        if query_t > label_t:
            label_tuple = dataset.get_node_label(query_t)
            if label_tuple is None:
                break
            label_ts, label_srcs, labels = (
                label_tuple[0],
                label_tuple[1],
                label_tuple[2],
            )
            label_ts = label_ts.numpy()
            label_srcs = label_srcs.numpy()
            labels = labels.numpy()
            label_t = dataset.get_label_time()

            preds = []

            for i in range(0, label_srcs.shape[0]):
                node_id = label_srcs[i]
                pred_vec = forecaster.query_dict(node_id)
                preds.append(pred_vec)
                forecaster.update_dict(node_id, labels[i])

            np_pred = np.stack(preds, axis=0)
            np_true = labels

            # print(np_true.shape, np_pred.shape)
            input_dict = {
                "y_true": np_true,
                "y_pred": np_pred,
                "eval_metric": [eval_metric],
            }
            result_dict = evaluator.eval(input_dict)
            score = result_dict[eval_metric]
            total_score += score
            num_label_ts += 1

    metric_dict = {}
    metric_dict[eval_metric] = total_score / num_label_ts 
    return metric_dict


raw file found, skipping download
Dataset directory is  /home/joe/anaconda3/lib/python3.11/site-packages/tgb/datasets/tgbn_trade
loading processed file


In [2]:
import sys
sys.path.append("/mnt/d/CodingArea/Python/TPPR-Node-Classification")
from utils.data_processing import Data
from utils.data_processing import TGB_load
from utils.my_dataloader import position_encoding
import numpy as np
from tgb.nodeproppred.dataset import NodePropPredDataset
from tgb.nodeproppred.dataset_pyg import PyGNodePropPredDataset
from tgb.nodeproppred.evaluate import Evaluator

/home/joe/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
name = "tgbn-genre"

dataset_py = PyGNodePropPredDataset(name=name, root=r"data/")

df, node_label_dict, edge_feat = dataset_py.dataset.generate_processed_files()
train_mask = dataset_py.train_mask
val_mask = dataset_py.val_mask
test_mask = dataset_py.test_mask
data_py = dataset_py.get_TemporalData()

train_data = data_py[train_mask]
val_data = data_py[val_mask]
test_data = data_py[test_mask]


graph_num = max(data_py.src.max().item(), data_py.dst.max().item())
edge_num = data_py.src.shape[0]

graph_feat = position_encoding(graph_num, emb_size=64).numpy()
train_data, val_data, test_data = TGB_load(train_data, val_data, test_data, graph_feat)

raw file found, skipping download
Dataset directory is  /home/joe/anaconda3/lib/python3.11/site-packages/tgb/data/tgbn_genre
loading processed file
loading processed file


In [ ]:
def batch_processor(data_label: dict[dict[int, np.ndarray]], data: Data)->list[tuple[np.ndarray]]:
  time_stamp = data.timestamps
  unique_ts = np.unique(time_stamp)
  idx_list = np.arange(time_stamp.shape[0])
  time_keys = list(data_label.keys())

  last_idx = 0
  batch_list: list[tuple] = list()
  for ts in unique_ts:
    time_mask = time_stamp==ts
    # time_mask[:last_idx] = False
    if ts not in time_keys:
      # print(ts)
      last_idx = idx_list[time_mask][-1]
      continue

    temp_dict = data_label[ts]
    keys = np.array(list(temp_dict.keys()))
    values = np.array(list(temp_dict.values()))

    unique_time_nodes = set(data.sources[time_mask]) | set(data.destinations[time_mask])
    if len(unique_time_nodes) != len(keys):
      print(f"At time {ts}; Under the same timetable the unique node {len(unique_time_nodes)} size and {len(set(keys))} isnt matched")
      print(f"The different unique nodes are {unique_time_nodes - set(keys)} \n")

    sort_idx = np.argsort(keys)
    sort_key = keys[sort_idx]
    values = values[sort_idx, :]
    last_idx = idx_list[time_mask][-1]

    backprop_mask = np.isin(np.array(sorted(unique_time_nodes)), sort_key)

    batch_list.append((backprop_mask, values, time_mask))
  print("process finshed")
  return batch_list

In [ ]:
batch_list = batch_processor(node_label_dict, train_data)

In [34]:
output_key = len(node_label_dict.keys())
time_stamp = train_data.timestamps
unique_ts = np.unique(time_stamp)

In [37]:
print(output_key, len(unique_ts))

1579 3061550


In [1]:
import pandas as pd

/home/joe/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
data = pd.read_csv(r"../../Toy_data.csv", encoding='latin1')